# PMAPS-MC-KR

#### Prototype, PMAPS analysis, Kripton MC data

In [ ]:
from __future__ import print_function

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np

In [ ]:
from invisible_cities.database import load_db
from invisible_cities.core.system_of_units_c import SystemOfUnits
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.core.wfm_functions as wfm
import invisible_cities.core.tbl_functions as tbl
import invisible_cities.core.peak_functions_c as cpf
import invisible_cities.core.pmaps_functions as pf
import invisible_cities.core.sensor_functions as sf
import invisible_cities.core.core_functions as cf

In [ ]:
import scipy.stats

### Kripton MC

1. P = 7 bar
2. V_gate = 7kV: E/P = 2
3. Processed by DIOMIRA

### Access to data base and definition of units

In [ ]:
DataPMT = load_db.DataPMT(0)
units = SystemOfUnits()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = load_db.DataSiPM(0)
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

### PMAPS file

In [ ]:
PMP_file = os.environ['IC_DATA']  + '/NEXT_v0_8_09/dst_NEXT_v0_08_09_Kr_ACTIVE_5_0_7bar_PMP_1000.root.h5'

### Read PMAP file

In [ ]:
t0 = time()
S1df, S2df, S2Sidf = pf.read_pmaps(PMP_file)
t1 = time()        
dt = t1 - t0
print("read PMPAS in {} s".format(dt))

In [ ]:
print('length of S1df = {} S2df = {} S2Sidf = {}'.format(
    len(S1df), len(S2df), len(S2Sidf)))

In [ ]:
S1df.head()

In [ ]:
plt.hist(S1df.peak[S1df.peak<5])

In [ ]:
plt.hist(S1df.time[S1df.peak==0]/units.mus)

In [ ]:
plt.hist(S1df.ene[S1df.peak==0], bins=20)

In [ ]:
plt.hist(S2df.peak[S2df.peak<5])

In [ ]:
plt.hist(S2df.time/units.mus)

In [ ]:
plt.hist(S2df.ene[S2df.ene<8000], bins=20)

### Comments:
1. S1 generated always at 100 mus. This is not realistic to compare with data (trigger on S2, fixed position is in S2, one has to scan a large window on S1).
2. S1 energy is small, peaking by the cut (0.25 pes), which may be unrealistic for data
3. Why the S2 distribtuion in time is not uniform? Why the concentrations of events is larger near the anode?
4. Why the large tail in energy (per bin) in S2, extending to 800 pes (total energy of S2 must be ~600 pes)

### Example waveforms

In [ ]:
pf.s12df_plot_waveforms(S1df, nmin=0, nmax=16, x=4, y=4)

In [ ]:
pf.s12df_plot_waveforms(S2df, nmin=0, nmax=16, x=4, y=4)

### SiPMs

In [ ]:
S2Sidf.head()

In [ ]:
def s2sipmdf_to_s2sipml(s2sipmdf, evt_max=10):
    """
    Accept a S2SiPMdf object (a S2Si pytable readout as a PD dataframe)
    and return a S12L dictionary
    """
    peak = s12df.peak.values.astype(np.int32)

    return cpm.cdf_to_dict(len(s12df.index), evt_max, s12df.event.values,
                           peak, s12df.time.values,   s12df.ene  .values)



In [ ]:
plt.hist(S2Sidf.nsipm, bins=20)

In [ ]:
plt.hist(S2Sidf.ene[S2Sidf.ene<80], bins=20)

In [ ]:
plt.hist(S2Sidf.nsample[S2Sidf.ene>0], bins=20)

In [ ]:
plt.hist(S2Sidf.ene[S2Sidf.nsample>10], bins=20)

### S12L

In [ ]:
evt=10000
t0 = time()
S1L = pf.s12df_to_s12l(S1df,evt_max=evt)
t1 = time()        
dt = t1 - t0

print("run {} evts in {} s, time/event = {}".format(evt, dt, dt/float(evt)))

In [ ]:
S1L[0]

In [ ]:
len(S1L)

In [ ]:
evt=10000
t0 = time()
S2L =  pf.s12df_to_s12l(S2df,evt_max=evt)
t1 = time()        
dt = t1 - t0

print("run {} evts in {} s, time/event = {}".format(evt, dt, dt/float(evt)))

In [ ]:
S2L[0]

In [ ]:
len(S2L)

In [ ]:
s1k = np.array(list(S1L.keys()))

In [ ]:
s2k = np.array(list(S2L.keys()))

In [ ]:
s12k=np.intersect1d(s1k,s2k)

In [ ]:
 df = np.setdiff1d(s1k,s2k)

In [ ]:
df

In [ ]:
df.max()

In [ ]:
cf.loc_elem_1d(df, df.max())

### S1 features (peak=0)

In [ ]:
evt=10000
t0 = time()
s1f = pf.s12_features(S1L, peak=0, max_events=evt)
t1 = time()        
dt = t1 - t0

print("run {} evts in {} s, time/event = {}".format(evt, dt, dt/float(evt)))

In [ ]:
plt.hist(s1f.w[s1f.w>0], bins=20)

In [ ]:
plt.hist(s1f.emax[s1f.emax>0], bins=20)

In [ ]:
plt.hist(s1f.tmax/units.mus, bins=20)

In [ ]:
plt.hist(s1f.w[s1f.tmax/units.mus>20], bins=20)

In [ ]:
plt.hist(s1f.etot[s1f.etot>0], bins=20)

In [ ]:
plt.hist(s1f.er[s1f.er>0], bins=10)

### S1 features (peak=1)

In [ ]:
evt=10000
t0 = time()
s1f = pf.s12_features(S1L, peak=1, max_events=evt)
t1 = time()        
dt = t1 - t0

print("run {} evts in {} s, time/event = {}".format(evt, dt, dt/float(evt)))

### S2 features (peak=0)

In [ ]:
evt=10000
t0 = time()
s2f = pf.s12_features(S2L, peak=0, max_events=evt)
t1 = time()        
dt = t1 - t0

print("run {} evts in {} s, time/event = {}".format(evt, dt, dt/float(evt)))

In [ ]:
plt.hist(s2f.w[s2f.w>0]/units.mus, bins=20)

In [ ]:
plt.hist(s2f.emax[s2f.emax>0], bins=20)

In [ ]:
plt.hist(s2f.tmax/units.mus, bins=20)

In [ ]:
plt.hist(s2f.etot[s2f.etot>0], bins=20)

In [ ]:
plt.hist(s2f.er[s2f.er>0], bins=20)

In [ ]:
class Pmap:
    """
    A Particle MAP (PMAP) is characterized by:
    2) peak maximum (both energy and time)
    3) energy total
    4) ratio peak/total energy
    """

    def __init__(self, length):
        self.w    = np.zeros(length, dtype=np.double)
        self.tmax = np.zeros(length, dtype=np.double)
        self.emax = np.zeros(length, dtype=np.double)
        self.etot = np.zeros(length, dtype=np.double)
        self.er   = np.zeros(length, dtype=np.double)

In [ ]:
def build_pmaps(S1L, S2L, S2Si, peak=0, max_events=100):
    """
    input: S1L
    returns a S1F object for specific peak
    """
    nk = np.array(list(S12L.keys()))

    # max event: but notice that some events may be missing
    evt_max = np.max(nk)

    n = min(evt_max+1, max_events)
    print('required {} events; found in dict {} events'
          .format(max_events, evt_max+1))
    s1f = S12F(n)

    for i in nk:
        if i >= n:
            break

        S1 = S12L[i]
        try:
            T = S1[peak][0]
            E = S1[peak][1]
        except KeyError:
            print('peak number {} does not exit in S12L'.format(peak))
            return 0

        s1f.w[i] = T[-1] - T[0]
        s1f.emax[i] = np.max(E)
        i_t = cf.loc_elem_1d(E, s1f.emax[i])
        s1f.tmax[i] = T[i_t]
        s1f.etot[i] = np.sum(E)

        if s1f.etot[i] > 0:
            s1f.er[i] = s1f.emax[i] / s1f.etot[i]
        else:
            s1f.er[i] = 0

    return s1f


In [ ]:
plt.scatter(s1e[s1e>0],s2e[s1e>0])

In [ ]:

t0 = time()
s1e, s1t = S12_get_ene_tpeak(S1df, event_list=s12k, peak=0)
t1 = time()        
dt = t1 - t0

print("run {} evts in {} s, time/event = {}".format(len(s12k), dt, dt/float(len(s12k))))

In [ ]:
plt.hist(s1t, bins=20)

In [ ]:
t0 = time()
s2e, s2t = S12_get_ene_tpeak(S2df, event_list=s12k, peak=0)
t1 = time()        
dt = t1 - t0

print("run {} evts in {} s, time/event = {}".format(len(s12k), dt, dt/float(len(s12k))))

In [ ]:
plt.hist((s2t-s1t)/units.mus, bins=20)

In [ ]:
dt = s2t-s1t

In [ ]:
plt.scatter(dt/units.mus, s1e)

In [ ]:
plt.scatter(dt, s2e)

In [ ]:
plt.scatter(s1e, s2e)

In [ ]:
def profile(x, y, bins=50, x_range=(0,1)):
    """
    Computes a profile plot 
    """

    means_result = scipy.stats.binned_statistic(x, [y, y**2], 
                                            bins=50, range=x_range, statistic='mean')
    means, means2 = means_result.statistic
    standard_deviations = np.sqrt(means2 - means**2)
    bin_edges = means_result.bin_edges
    bin_centers = (bin_edges[:-1] + bin_edges[1:])/2.

    plt.errorbar(x=bin_centers, y=means, yerr=standard_deviations, 
                 linestyle='none', marker='.')
    plt.show()

In [ ]:
profile(dt/units.mus, s1e, bins=25, x_range=(0,600))

In [ ]:
profile(dt/units.mus, s2e, bins=25, x_range=(0,600))

In [ ]:
profile(s1e, s2e, bins=25, x_range=(0,25))

In [ ]:
def S1S2_ana(S1L, S2L):
    """
    Consider events with 1 S1 and 1 S2
    Compute t0
    fill vectors for es1, es2, ds12
    """
    s1k = np.array(S1L.keys())
    s2k = np.array(S2L.keys())
    s12k=np.intersect1d(s1k,s2k)
    
    es1 = S12_get_ene_inc(S1df, event_list=s12k, peak=0)
    es2 = S12_get_ene_inc(S2df, event_list=s12k, peak=0)
        
    